In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import os

In [32]:
%pwd

'c:\\Users\\Akhil\\OneDrive\\Desktop\\animal-disease-classification-with-llm'

In [33]:
os.chdir(r"\Users\Akhil\OneDrive\Desktop\animal-disease-classification-with-llm")

In [34]:
%pwd

'c:\\Users\\Akhil\\OneDrive\\Desktop\\animal-disease-classification-with-llm'

In [35]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
 

In [36]:
from classifier.constants import *
from classifier.utils.common import read_yaml, create_directories

In [37]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    # defining the dataingestion config as the return type
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir),
        )

        return data_ingestion_config
    
    

In [38]:
import os
import urllib.request as request
import zipfile
from classifier import logger
from classifier.utils.common import get_size

In [39]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) -> Path:
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                self.config.source_URL, self.config.local_data_file
            )
            logger.info(
                f"File : {filename} downloaded with following info: \n{headers}"
            )
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
            )

    def extract_zip_file(self) -> None:
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)  
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [40]:
# Now we create the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-18 10:44:01,163 - INFO - common - yaml file: config\config.yaml loaded successfully]
[2026-01-18 10:44:01,170 - INFO - common - yaml file: params.yaml loaded successfully]
created directory at: artifacts
created directory at: artifacts/data_ingestion
[2026-01-18 10:44:04,747 - INFO - 831689671 - File : artifacts\data_ingestion\data.zip downloaded with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A573:F9C83:265120:62157C:696C6C1C
Accept-Ranges: bytes
Date: Sun, 18 Jan 2026 05:14:06 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830030-CCU
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1768713246.591984,VS0,VE963
Vary: A